In [ ]:
# Import the required libraries and dependencies
import pandas as pd
import requests
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
song = input("What song are you looking for?")

In [ ]:
url = f"http://127.0.0.1:5501/api/v1.0/trackrec/{song}/"

In [ ]:
response = requests.get(url)
if response.status_code ==200:
    data = response.json()
else:
    print(f"API failed")

In [ ]:
new_df = pd.DataFrame(data)
new_df = new_df.set_index('song')
new_df

In [ ]:
new_df.columns

In [ ]:
scaler = StandardScaler()

In [ ]:
song_scaler = scaler.fit(new_df[['acousticness', 'danceability', 'duration', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'popularity', 'speechiness',
       'tempo', 'valence']])

In [ ]:
song_data_scaled = song_scaler.transform(new_df[['acousticness', 'danceability', 'duration', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'popularity', 'speechiness',
       'tempo', 'valence']])

In [ ]:
df_song_scaled = pd.DataFrame(song_data_scaled, columns= ['acousticness', 'danceability', 'duration', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'popularity', 'speechiness',
       'tempo', 'valence'])
df_song_scaled['song'] = new_df.index
df_song_scaled = df_song_scaled.set_index('song')
df_song_scaled

In [ ]:
model = KMeans(n_clusters=10)
model.fit(df_song_scaled)
song_clusters = model.predict(df_song_scaled)

In [ ]:
df_song_predictions = df_song_scaled.copy()
df_song_predictions['cluster'] = song_clusters
df_song_predictions

In [ ]:
url2 = f"http://127.0.0.1:5501/api/v1.0/track/{song}"
response2 = requests.get(url2)
if response2.status_code ==200:
    data2 = response2.json()
else:
    print(f"API failed, {response2.status_code}")

new_df2 = pd.DataFrame(data2)
new_df2 = new_df2.set_index('song')
new_df2

In [ ]:
new_song_data_scaled = song_scaler.transform(new_df2[['acousticness', 'danceability', 'duration', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'popularity', 'speechiness',
       'tempo', 'valence']])

In [ ]:
new_df_song_scaled = pd.DataFrame(new_song_data_scaled, columns= ['acousticness', 'danceability', 'duration', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'popularity', 'speechiness',
       'tempo', 'valence'])
new_df_song_scaled

In [ ]:
new_df_song_scaled['song'] = new_df2.index
new_df_song_scaled = new_df_song_scaled.set_index('song')
new_df_song_scaled

In [ ]:
prediction = model.predict(new_df_song_scaled)
print(int(prediction))

In [ ]:
predicted_songs = df_song_predictions.loc[df_song_predictions['cluster']==int(prediction)]

In [ ]:
predicted_songs

In [ ]:
# Create the PCA model instance where n_components=2
pca = PCA(n_components=2)

In [ ]:
# Fit the df_stocks_scaled data to the PCA
spotify_pca_data = pca.fit_transform(predicted_songs)

# Review the first five rose of the PCA data
# using bracket notation ([0:5])
spotify_pca_data[:5]

In [ ]:
# Calculate the explained variance
pca.explained_variance_ratio_

In [ ]:
# Creating a DataFrame with the PCA data
spotify_pca_df = pd.DataFrame(spotify_pca_data, columns=["PC1", "PC2"])

# Copy the tickers names from the original data
spotify_pca_df["name"] = new_df2["name"]

# Set the Ticker column as index
spotify_pca_df = spotify_pca_df.set_index("name")

# Review the DataFrame
spotify_pca_df.head()

In [ ]:
# Initialize the K-Means model with n_clusters=3
model = KMeans(n_clusters=5)

# Fit the model for the df_stocks_pca DataFrame
model.fit(spotify_pca_df)

# Predict the model segments (clusters)
song_clusters = model.predict(spotify_pca_df)

# Print the stock segments
print(song_clusters)

In [ ]:
# Create a copy of the df_stocks_pca DataFrame and name it as df_stocks_pca_predictions
spotify_pca_df_predictions = spotify_pca_df.copy()

# Create a new column in the DataFrame with the predicted clusters
spotify_pca_df_predictions["SongCluster"] = song_clusters

# Review the DataFrame
spotify_pca_df_predictions.head()

In [ ]:
# Create the scatter plot with x="PC1" and y="PC2"
spotify_pca_df_predictions.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="SongCluster",
    title = "Scatter Plot by Song Segment - PCA=2"
)

In [ ]:
artist_list =new_df['artist']

In [ ]:
merged_list = pd.merge(predicted_songs, artist_list, how ='left', on= 'song')

In [ ]:
merged_list['artist']